In [1]:
# Google Colab compatibility
project_folder = ''

In [2]:
from jones_kording_jupyter.custompackage.load_data import *
from jones_kording_jupyter.custompackage.load_architecture import *
from jones_kording_jupyter.custompackage.traintestloop import *
import numpy as np

In [3]:
# use_cuda = torch.cuda.is_available()
use_cuda = False

# Initialize settings
bs = 256
weighting = 'paired'
trials = 10
epochs = 2000
trees_set = [1]

# Load class-dataset list
classes = np.load(project_folder + 'results/classes.npy', allow_pickle=True)

# Initialize final test loss and accuracy variables
loss = np.zeros((len(classes), trials, len(trees_set)))
acc = np.zeros((len(classes), trials, len(trees_set)))

# For each dataset enumerated from classes list
for j, (t1, t2, ds) in enumerate(classes):

    if ds in ['svhn', 'cifar10']:
        continue

    print(t1, t2, ds)
    # Load data loaders
    trainloaders, validloaders, testloader = dataset_weighted_split_all(bs, t1, t2, weighting, trials, ds, permute=False)
    # Initialize input size for model initialization purposes
    input_size = trainloaders[0].dataset.tensors[0][0].shape[0]
    # For each trial
    for k, trees in enumerate(trees_set):
        # For every k-tree defined by trees_set
        for i in range(trials):
            print(j, i, k)
            # Initialize the ktree model
            if use_cuda:
                model = ktree_gen(ds=ds, Repeats=trees, Padded=True).cuda()
            else:
                model = ktree_gen(ds=ds, Repeats=trees, Padded=True)

            #Train and test ktree, assigning loss and acc values
            loss_curve, acc_curve, loss[j,i,k], acc[j,i,k], model_t = train_test_ktree(model, trainloaders[i],
                                                                                  validloaders[i], testloader, epochs = epochs, randorder=False, use_cuda=use_cuda)
            # Save accuracy and loss arrays
            np.save(project_folder + 'results/jk_1tree_acc.npy', acc)
            np.save(project_folder + 'results/jk_1tree_loss.npy', loss)

3 5 mnist
0 0 0
Early stopping
Finished Training, 1247 epochs
Accuracy of the network on the test images: 86.225026 %
0 1 0
Early stopping
Finished Training, 486 epochs
Accuracy of the network on the test images: 76.340694 %
0 2 0
Early stopping
Finished Training, 294 epochs
Accuracy of the network on the test images: 78.496320 %
0 3 0
Early stopping
Finished Training, 874 epochs
Accuracy of the network on the test images: 87.697161 %
0 4 0
Early stopping
Finished Training, 401 epochs
Accuracy of the network on the test images: 75.709779 %
0 5 0
Early stopping
Finished Training, 1208 epochs
Accuracy of the network on the test images: 61.409043 %
0 6 0
Early stopping
Finished Training, 621 epochs
Accuracy of the network on the test images: 90.378549 %
0 7 0
Early stopping
Finished Training, 307 epochs
Accuracy of the network on the test images: 78.864353 %
0 8 0
Early stopping
Finished Training, 754 epochs
Accuracy of the network on the test images: 88.065195 %
0 9 0
Early stopping
Fini

KeyboardInterrupt: 

In [ ]:
class_set = 0
acc = np.load(project_folder + 'results/ktree_acc_orig_'+str(class_set)+'.npy', allow_pickle=True)
print("RESULTS:")
for j, (t1, t2, ds) in enumerate(classes):
    print(f"Dataset: {ds} / Pair: {t1}-{t2}")
    for k, trees in enumerate(trees_set):
        print(f"{trees}-tree")
        print(f"Accuracy: {round(np.mean(acc[j,:,k]), 4)} ± {round(np.std(acc[j,:,k]), 4)}")